https://zzaebok.github.io/deep_learning/nlp/Bert-for-classification/

In [1]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from pytorch_transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from torch.optim import Adam
import torch.nn.functional as F

In [2]:
train_df = pd.read_csv('./nsmc/ratings_train.txt', sep='\t')
test_df = pd.read_csv('./nsmc/ratings_test.txt', sep='\t')

In [3]:
train_df.dropna(inplace=True)
test_df.dropna(inplace=True)

train_df = train_df.sample(frac=0.4, random_state=999)
test_df = test_df.sample(frac=0.4, random_state=999)

In [4]:
class NsmcDataset(Dataset):
    ''' Naver Sentiment Movie Corpus Dataset '''
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.df.iloc[idx, 1]
        label = self.df.iloc[idx, 2]
        return text, label

In [5]:
nsmc_train_dataset = NsmcDataset(train_df)
train_loader = DataLoader(nsmc_train_dataset, batch_size=3, shuffle=True, num_workers=2)

In [6]:
device = torch.device("cuda")
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [7]:
nsmc_train_dataset = NsmcDataset(train_df)
train_loader = DataLoader(nsmc_train_dataset, batch_size=3, shuffle=True)

In [8]:
optimizer = Adam(model.parameters(), lr=1e-6)

itr = 1
p_itr = 500
epochs = 2
total_loss = 0
total_len = 0
total_correct = 0


model.train()
for epoch in range(epochs):
    
    for text, label in train_loader:
        optimizer.zero_grad()
        
        # encoding and zero padding
        encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
        padded_list =  [e + [0] * (512-len(e)) for e in encoded_list]
        
        sample = torch.tensor(padded_list)
        sample, label = sample.to(device), label.to(device)
        labels = torch.tensor(label)
        outputs = model(sample, labels=labels)
        loss, logits = outputs

        pred = torch.argmax(F.softmax(logits), dim=1)
        correct = pred.eq(labels)
        total_correct += correct.sum().item()
        total_len += len(labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        
        if itr % p_itr == 0:
            print('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch+1, epochs, itr, total_loss/p_itr, total_correct/total_len))
            total_loss = 0
            total_len = 0
            total_correct = 0

        itr+=1

C:\Users\jinma\AppData\Local\Continuum\anaconda3\envs\basic\lib\site-packages\ipykernel_launcher.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\jinma\AppData\Local\Continuum\anaconda3\envs\basic\lib\site-packages\ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[Epoch 1/2] Iteration 500 -> Train Loss: 0.6955, Accuracy: 0.523
[Epoch 1/2] Iteration 1000 -> Train Loss: 0.6950, Accuracy: 0.527
[Epoch 1/2] Iteration 1500 -> Train Loss: 0.6915, Accuracy: 0.534
[Epoch 1/2] Iteration 2000 -> Train Loss: 0.6951, Accuracy: 0.514
[Epoch 1/2] Iteration 2500 -> Train Loss: 0.6937, Accuracy: 0.521
[Epoch 1/2] Iteration 3000 -> Train Loss: 0.6826, Accuracy: 0.554
[Epoch 1/2] Iteration 3500 -> Train Loss: 0.6192, Accuracy: 0.672
[Epoch 1/2] Iteration 4000 -> Train Loss: 0.5957, Accuracy: 0.692
[Epoch 1/2] Iteration 4500 -> Train Loss: 0.5835, Accuracy: 0.697
[Epoch 1/2] Iteration 5000 -> Train Loss: 0.5588, Accuracy: 0.721
[Epoch 1/2] Iteration 5500 -> Train Loss: 0.5422, Accuracy: 0.728
[Epoch 1/2] Iteration 6000 -> Train Loss: 0.5266, Accuracy: 0.746
[Epoch 1/2] Iteration 6500 -> Train Loss: 0.5336, Accuracy: 0.733
[Epoch 1/2] Iteration 7000 -> Train Loss: 0.5060, Accuracy: 0.754
[Epoch 1/2] Iteration 7500 -> Train Loss: 0.5055, Accuracy: 0.758
[Epoch 1/2]

In [9]:
print("Done!")

Done!
